In [5]:
import torch
from torch.cuda.amp import autocast

# Feature Extractor

In [3]:
!export CUDA_HOME=/usr/local/cuda-11.7
!export LD_LIBRARY_PATH=/usr/local/cuda-11.7/lib
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_May__3_18:49:52_PDT_2022
Cuda compilation tools, release 11.7, V11.7.64
Build cuda_11.7.r11.7/compiler.31294372_0


In [2]:
from model.pvcnn_generation import PVCNNFeatures
import torch

Using /home/mathi/.cache/torch_extensions/py310_cu117 as PyTorch extensions root...
No modifications detected for re-loaded extension module _pvcnn_backend, skipping build step...
Loading extension module _pvcnn_backend...


In [3]:
model = PVCNNFeatures(input_dim=3, extra_feature_channels=0).cuda()

In [14]:
test = torch.randn(3, 3, 2048*8)

In [15]:
feats = model(test.cuda())

In [16]:
for f in feats:
    print(f.shape)

torch.Size([3, 512, 16])
torch.Size([3, 256, 64])
torch.Size([3, 256, 256])
torch.Size([3, 128, 1024])


# RIN

In [3]:
from model.recurrent_interface_network import RIN

# Gridding

In [6]:
import open3d as o3d
import numpy as np

cloud = "../3d/421378.ply"

pc = o3d.io.read_point_cloud(cloud)
pcnpy = np.asarray(pc.points)

KeyboardInterrupt: 

In [ ]:
pcnpy.shape

In [ ]:
point_cloud = pcnpy
# substract center
point_cloud -= np.mean(point_cloud, axis=0)

In [ ]:
results[3]

# Datasets

In [2]:
!export CUDA_HOME=/usr/local/cuda-11.7

In [3]:
from data.dataloader import get_dataloader
from omegaconf import OmegaConf

Using /home/mathi/.cache/torch_extensions/py310_cu117 as PyTorch extensions root...
No modifications detected for re-loaded extension module _pvcnn_backend, skipping build step...
Loading extension module _pvcnn_backend...


In [4]:
conf = "configs/overfit_room.yml"
opt = OmegaConf.load(conf)
opt.data.dataset = "Indoor"
opt.data.npoints = 2**14
opt.distribution_type = "single"
dl, *_ = get_dataloader(opt)

2023-10-12 17:10:20.070 | INFO     | data.dataloader:get_dataloader:43 - Loading IndoorScenes dataset, which is currently only for overfitting on one scene!


2023-10-12 17:10:25.198 | INFO     | data.arkit:__init__:58 - Downsampled to voxel size: 0.01
2023-10-12 17:10:25.199 | INFO     | data.arkit:__init__:59 - Number of points in arkit: 525143
2023-10-12 17:10:25.199 | INFO     | data.arkit:__init__:60 - Number of points in faro: 1184923


In [7]:
batch = next(iter((dl)))
data = batch["train_points"]
lowres = batch["train_points_lowres"]
data.shape, data.min(), data.max(), data.mean(), data.std()

(torch.Size([32, 16384, 3]),
 tensor(-0.9696),
 tensor(0.4939),
 tensor(-0.0037),
 tensor(0.2517))

In [8]:
import pyviz3d.visualizer as viz

ps = 2

v = viz.Visualizer()
v.add_points("hires", data[1, ...].numpy(), point_size=ps, visible=False)
v.add_points("lowres", lowres[1, ...].numpy(), point_size=ps, visible=True)
v.save("test")


************************************************************************
1) Start local server:
    cd /mnt/c/Users/mathi/Documents/GitHub/DPU/PCUpsampling/test; python -m http.server 6008
2) Open in browser:
    http://0.0.0.0:6008
************************************************************************


In [ ]:
from data.utils import concat_nn
import torch

In [ ]:
%%time
x1 = torch.randn(100000, 3)
x2 = torch.randn(100000, 3)
y = concat_nn(x1, x2)

In [ ]:
model = PVCLion(out_dim=3, input_dim=3, embed_dim=16, npoints=1_000_000, use_att=False, extra_feature_channels=3, dropout=0.1).cuda()
# print number of parameters
print(f"Number of parameters: {sum(p.numel() for p in model.parameters()) / 1_000_000}M")

In [ ]:
class DotDict(dict):
    """
    a dictionary that supports dot notation 
    as well as dictionary access notation 
    usage: d = DotDict() or d = DotDict({'val1':'first'})
    set attributes: d.val2 = 'second' or d['val2'] = 'second'
    get attributes: d.val2 or d['val2']
    """
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

    def __init__(self, dct):
        for key, value in dct.items():
            if hasattr(value, 'keys'):
                value = DotDict(value)
            self[key] = value

In [ ]:
from minkowski.minkunet import ResUNet14, TinyUnet

config = DotDict
config.bn_momentum = 0.9
config.conv1_kernel_size = 7

model = TinyUnet(in_channels=3, out_channels=3, config=config, D=3).cuda()
print(f"Number of parameters: {sum(p.numel() for p in model.parameters()) / 1_000_000}M")

In [ ]:
ds = IndoorScenes(root_dir="../../3d/", npoints=1_000_000)

In [ ]:
data = ds[0]

pcond = data["train_points_lowres"].unsqueeze(0).cuda()
noise = torch.randn(1, 1_000_000, 3).cuda()
t = torch.rand(1).cuda()

# concat
#x = torch.cat([pcond, noise], dim=1)

In [ ]:
import MinkowskiEngine as ME

def to_parse_tensor(coords, feats=None, voxel_size=0.001):
    """
    Converts the input coordinates and features to a sparse tensor.

    Args:
        coords (torch.Tensor): The input coordinates.
        feats (torch.Tensor): The input features.
        voxel_size (float): The voxel size for scaling the coordinates.

    Returns:
        ME.SparseTensor: The sparse tensor with the converted coordinates and features.
    """
    # scaling
    coords = coords / voxel_size
    b, n, c = coords.shape
    
    if feats is None:
        feats = torch.zeros((b*n, 3)).to(coords.device)
    elif feats.ndim == 3:
        feats = feats.reshape(b*n, 3)

    stensor = ME.SparseTensor(
        features=feats, # Convert to a tensor
        coordinates=ME.utils.batched_coordinates([c for c in coords], dtype=torch.float32, device=coords.device),  # coordinates must be defined in a integer grid. If the scale
        quantization_mode=ME.SparseTensorQuantizationMode.UNWEIGHTED_AVERAGE  # when used with continuous coordinates, average features in the same coordinate
    )
    return stensor

In [ ]:
xs = to_parse_tensor(coords=noise, feats=noise, voxel_size=0.001)

In [ ]:
model(xs)

In [ ]:
# voxelization
from modules.voxelization import Voxelization
from modules import functional as F
import torch
from torch import nn

In [ ]:
testhr = torch.rand(1, 3, 2048).cuda()
testlr = testhr[:, :, ::128]

resolution = 4

vox = Voxelization(resolution=resolution, normalize=True, eps=0)


voxfhr, norm_coords_hr = vox(features=testhr, coords=testhr)
voxflr, norm_coords_lr = vox(features=testlr, coords=testlr)

devoxhr = F.trilinear_devoxelize(voxfhr, norm_coords_hr, resolution)
devoxlr = F.trilinear_devoxelize(voxflr, norm_coords_hr, resolution)


In [ ]:
voxfhr.shape, norm_coords_hr.shape

In [ ]:
class FeatureVoxelConcatenation(nn.module):
    def __init__(self, resolution):
        super().__init__()
        self.resolution = resolution
        self.vox = Voxelization(resolution=resolution, normalize=True, eps=0)

    def forward(self, x1_features, x2_features, x1_coords, x2_coords):
        vox_x1, nc_x1 = self.vox(features=x1_features, coords=x1_coords)
        vox_x2, nc_x2 = self.vox(features=x2_features, coords=x2_coords)

        devox_mixed = F.trilinear_devoxelize(vox_x2, nc_x1, self.resolution)

        return torch.cat([x1_features, devox_mixed], dim=1)


In [ ]:
devoxlr.shape

In [ ]:
voxflr.shape, voxfhr.shape

In [ ]:
voxchr

In [ ]:
test_vox.__len__()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
with autocast(cache_enabled=False):
    out = model(inputs=x, t=t)

# Torchsparse

In [ ]:
from torch import nn
import torch
from torchsparse import nn as spnn
from torchsparse.utils.quantize import sparse_quantize
from torchsparse.utils.collate import sparse_collate_fn

from torchsparse import SparseTensor

class TestModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
    
        self.net = nn.Sequential(
            spnn.Conv3d(3, 64, 3),
            spnn.BatchNorm(64),
            spnn.ReLU(True),
            nn.Linear(64, 64),
        )
    
    def forward(self, x):
        x = self.net(x)
        return x

model = nn.Sequential(
        spnn.Conv3d(3, 32, 3),
        spnn.BatchNorm(32),
        spnn.ReLU(True),
        spnn.Conv3d(32, 64, 2, stride=2),
        spnn.BatchNorm(64),
        spnn.ReLU(True),
        spnn.Conv3d(64, 64, 2, stride=2, transposed=True),
        spnn.BatchNorm(64),
        spnn.ReLU(True),
        spnn.Conv3d(64, 32, 3),
        spnn.BatchNorm(32),
        spnn.ReLU(True),
        spnn.Conv3d(32, 10, 1),
    ).to("cuda")



In [ ]:
test_in = torch.randn(2048, 3).numpy()
coords, indices = sparse_quantize(test_in, 1e-3, return_index=True)
coords = torch.tensor(coords, dtype=torch.int)
feats = torch.tensor(test_in[indices], dtype=torch.float)
tensor = SparseTensor(coords=coords, feats=feats)

model(tensor)